
# 목표: 우리는 어떤 사람이 선거 투표에 참여하는지, 심리적 성향과 인구통계학적 정보를 가지고 예측할 수 있을까?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder, StandardScaler
import lightgbm as lgbm

In [2]:
#데이터 불러오기
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE','QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)

<h1>데이터 전처리</h1>

---

#### 1.데이터 확인

In [3]:
#DataFrame.isnull()을 통해 train, test 데이터에는 결손값이 없는 것을 알 수 있음.
print("train 데이터 누락값 수: {}개".format(train.isnull().sum().sum()))
print("test 데이터 누락값 수: {}개".format(test.isnull().sum().sum()))

train 데이터 누락값 수: 0개
test 데이터 누락값 수: 0개


#### 2.결손값 처리

In [4]:
# 'education', 'engnat', 'hand', 'urban' 컬럼에 있는 무응답 0을 제거
zero_features = ['education', 'engnat', 'hand', 'urban']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        print("{} column에서 무응답이 있는 행 {}개 삭제  ".format(col,df[df[col]==0].shape[0]))
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df

#train, test에서 결손값 제거
print('train데이터 결손값 삭제')
train = drop_NoResponse(train)
print('\ntest데이터 결손값 삭제')
test = drop_NoResponse(test)

train데이터 결손값 삭제
education column에서 무응답이 있는 행 432개 삭제  
engnat column에서 무응답이 있는 행 62개 삭제  
hand column에서 무응답이 있는 행 126개 삭제  
urban column에서 무응답이 있는 행 241개 삭제  

test데이터 결손값 삭제
education column에서 무응답이 있는 행 96개 삭제  
engnat column에서 무응답이 있는 행 14개 삭제  
hand column에서 무응답이 있는 행 25개 삭제  
urban column에서 무응답이 있는 행 58개 삭제  


#### 3.이상값 처리

In [5]:
#train데이터의 "familysize"column은 2번째로 큰 값이 44이고 제일 큰 값이 999일 정도로 outlier가 존재함을 알 수 있음.
print(train['familysize'].value_counts(ascending=False))

2      14003
3       8815
1       5141
4       3821
5       1483
0        946
6        647
7        309
8        172
9         96
10        47
11        29
12        16
13        10
14         7
15         7
44         3
17         3
16         2
20         2
21         1
999        1
34         1
18         1
30         1
Name: familysize, dtype: int64


In [6]:
#outlier 값 출력 함수

# def get_outlier(df=None, column=None, weight=1.5):
#     fraud = df[column]
#     quantile_25 = np.percentile(fraud.values, 25)
#     quantile_75 = np.percentile(fraud.values, 75)
#     iqr = quantile_75 - quantile_25
#     iqr_weight = iqr * weight
#     lowest_val = quantile_25 - iqr_weight
#     highest_val = quantile_75 + iqr_weight
#     print("iqr:{}\nquantile_25={}\nquantile_75={}\nlowest_val={}\nhighest_val={}".format(iqr, quantile_25, quantile_75, lowest_val, highest_val))

In [7]:
#IQR(Inter Quantile Range)을 이용해 이상값을 처리하는 것보단 제일 큰 값인 999만 처리하는 것이 데이터적 측면에서 훨씬 탁월하다고 생각하여 max값만 제거함.
max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
train

,QaA,QbA,QcA,QdA,QeA,QfA,QgA,QhA,QiA,QjA,QkA,QlA,QmA,QnA,QoA,QpA,QqA,QrA,QsA,QtA,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,1.0,5.0,5.0,1.0,1.0,1.0,5.0,5.0,2.0,5.0,2.0,5.0,4.0,1.0,5.0,4.0,1.0,1.0,5.0,5.0,20s,3,1,2,Male,1,1,White,Christian_Catholic,2,2,1,5,2,4,5,4,1,2,2,1,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,1
1,1.0,5.0,5.0,1.0,1.0,3.0,1.0,4.0,5.0,5.0,2.0,5.0,5.0,1.0,5.0,5.0,2.0,1.0,3.0,5.0,10s,2,2,4,Male,1,1,Asian,Hindu,1,0,2,2,0,6,1,5,5,6,3,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1
2,5.0,1.0,2.0,1.0,3.0,5.0,4.0,3.0,5.0,3.0,5.0,1.0,1.0,5.0,1.0,1.0,2.0,5.0,1.0,4.0,20s,2,1,3,Female,1,1,White,Christian_Other,0,1,0,5,3,6,0,2,1,6,2,1,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0
3,4.0,1.0,1.0,4.0,1.0,3.0,4.0,1.0,5.0,1.0,5.0,1.0,1.0,4.0,4.0,5.0,4.0,4.0,2.0,5.0,10s,3,2,3,Male,1,1,Other,Hindu,2,2,4,5,0,1,0,2,0,4,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
4,4.0,5.0,5.0,3.0,1.0,1.0,1.0,2.0,1.0,5.0,3.0,5.0,4.0,2.0,5.0,1.0,4.0,4.0,4.0,5.0,20s,2,1,4,Female,1,1,White,Other,6,3,6,2,0,6,2,0,6,4,2,1,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,1.0,5.0,5.0,1.0,1.0,1.0,3.0,3.0,1.0,5.0,3.0,5.0,4.0,3.0,5.0,5.0,1.0,1.0,5.0,5.0,20s,2,1,6,Female,1,1,Black,Other,4,3,3,5,0,1,2,2,0,3,2,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1
36421,3.0,4.0,5.0,2.0,1.0,3.0,1.0,3.0,4.0,5.0,4.0,2.0,5.0,2.0,4.0,5.0,3.0,2.0,5.0,5.0,10s,1,2,4,Male,3,1,White,Jewish,5,1,1,6,2,5,4,6,3,6,1,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1
36422,2.0,4.0,2.0,1.0,2.0,1.0,4.0,2.0,2.0,4.0,5.0,5.0,4.0,1.0,5.0,2.0,4.0,1.0,5.0,3.0,30s,4,1,6,Male,1,2,White,Agnostic,1,2,0,5,1,2,1,2,1,5,2,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1
36423,5.0,1.0,1.0,3.0,5.0,1.0,4.0,1.0,4.0,2.0,5.0,5.0,4.0,5.0,2.0,1.0,5.0,1.0,2.0,4.0,40s,4,1,4,Female,3,1,White,Other,4,4,5,4,1,3,2,5,1,5,3,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1


#### 4. 데이터 변환

In [8]:
# label_encoding 사용시
#'gender', 'age_group', 'race', 'religion'의 데이터는 정수형이 아니기 때문에 인코딩을 통해 데이터를 정수형으로 변환

# def label_encoding(df):
#     le = LabelEncoder()
#     original_columns = list(df.columns)
    
#     for col in df:
#         # df의 컬럼의 유형이 object인 것들만
#         if df[col].dtype == 'object':
#             le.fit(df[col])
#             df[col] = le.transform(df[col])

#     # 새롭게 만들어진 컬럼들의 이름을 리스트로 저장
#     new_columns = [c for c in df.columns if c not in original_columns]
    
#     # 수치형으로 변경된 df와 새롭게 만들어진 컬럼 이름 리스트를 반환
#     return df, new_columns

In [9]:
#'gender', 'age_group', 'race', 'religion'의 데이터는 정수형이 아니기 때문에 인코딩을 통해 데이터를 정수형으로 변환
train = pd.get_dummies(train)
train

,QaA,QbA,QcA,QdA,QeA,QfA,QgA,QhA,QiA,QjA,QkA,QlA,QmA,QnA,QoA,QpA,QqA,QrA,QsA,QtA,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,age_group_+70s,age_group_10s,age_group_20s,age_group_30s,age_group_40s,age_group_50s,age_group_60s,gender_Female,gender_Male,race_Arab,race_Asian,race_Black,race_Indigenous Australian,race_Native American,race_Other,race_White,religion_Agnostic,religion_Atheist,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
0,1.0,5.0,5.0,1.0,1.0,1.0,5.0,5.0,2.0,5.0,2.0,5.0,4.0,1.0,5.0,4.0,1.0,1.0,5.0,5.0,3,1,2,1,1,2,2,1,5,2,4,5,4,1,2,2,1,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1,1.0,5.0,5.0,1.0,1.0,3.0,1.0,4.0,5.0,5.0,2.0,5.0,5.0,1.0,5.0,5.0,2.0,1.0,3.0,5.0,2,2,4,1,1,1,0,2,2,0,6,1,5,5,6,3,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,5.0,1.0,2.0,1.0,3.0,5.0,4.0,3.0,5.0,3.0,5.0,1.0,1.0,5.0,1.0,1.0,2.0,5.0,1.0,4.0,2,1,3,1,1,0,1,0,5,3,6,0,2,1,6,2,1,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3,4.0,1.0,1.0,4.0,1.0,3.0,4.0,1.0,5.0,1.0,5.0,1.0,1.0,4.0,4.0,5.0,4.0,4.0,2.0,5.0,3,2,3,1,1,2,2,4,5,0,1,0,2,0,4,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,4.0,5.0,5.0,3.0,1.0,1.0,1.0,2.0,1.0,5.0,3.0,5.0,4.0,2.0,5.0,1.0,4.0,4.0,4.0,5.0,2,1,4,1,1,6,3,6,2,0,6,2,0,6,4,2,1,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,1.0,5.0,5.0,1.0,1.0,1.0,3.0,3.0,1.0,5.0,3.0,5.0,4.0,3.0,5.0,5.0,1.0,1.0,5.0,5.0,2,1,6,1,1,4,3,3,5,0,1,2,2,0,3,2,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
36421,3.0,4.0,5.0,2.0,1.0,3.0,1.0,3.0,4.0,5.0,4.0,2.0,5.0,2.0,4.0,5.0,3.0,2.0,5.0,5.0,1,2,4,3,1,5,1,1,6,2,5,4,6,3,6,1,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
36422,2.0,4.0,2.0,1.0,2.0,1.0,4.0,2.0,2.0,4.0,5.0,5.0,4.0,1.0,5.0,2.0,4.0,1.0,5.0,3.0,4,1,6,1,2,1,2,0,5,1,2,1,2,1,5,2,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
36423,5.0,1.0,1.0,3.0,5.0,1.0,4.0,1.0,4.0,2.0,5.0,5.0,4.0,5.0,2.0,1.0,5.0,1.0,2.0,4.0,4,1,4,3,1,4,4,5,4,1,3,2,5,1,5,3,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [10]:
test = pd.get_dummies(test)
test

,QaA,QbA,QcA,QdA,QeA,QfA,QgA,QhA,QiA,QjA,QkA,QlA,QmA,QnA,QoA,QpA,QqA,QrA,QsA,QtA,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,age_group_+70s,age_group_10s,age_group_20s,age_group_30s,age_group_40s,age_group_50s,age_group_60s,gender_Female,gender_Male,race_Arab,race_Asian,race_Black,race_Indigenous Australian,race_Native American,race_Other,race_White,religion_Agnostic,religion_Atheist,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
0,1.0,4.0,5.0,2.0,5.0,1.0,2.0,3.0,5.0,4.0,4.0,3.0,3.0,3.0,3.0,5.0,2.0,2.0,4.0,5.0,2,1,1,1,1,4,2,1,3,0,2,5,5,0,6,3,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
1,2.0,2.0,3.0,2.0,2.0,1.0,3.0,1.0,3.0,5.0,3.0,5.0,4.0,1.0,4.0,3.0,1.0,2.0,3.0,3.0,4,1,2,1,2,5,4,1,5,2,3,0,5,0,6,3,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,2.0,1.0,1.0,3.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,2.0,4.0,2.0,1.0,5.0,5.0,3.0,1.0,2,1,2,1,1,2,0,2,1,0,0,1,3,4,5,2,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,2.0,5.0,2.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,2.0,5.0,2.0,3.0,2.0,1.0,5.0,2,2,1,1,1,1,6,0,6,0,3,4,6,0,6,3,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,1.0,4.0,4.0,1.0,1.0,4.0,4.0,2.0,5.0,4.0,4.0,5.0,5.0,4.0,2.0,4.0,2.0,3.0,5.0,3.0,2,1,2,2,1,4,2,2,1,0,1,1,3,3,5,1,0,0,1,1,1,0,1,1,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9102,1.0,5.0,5.0,1.0,1.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,4.0,3.0,5.0,3,2,2,1,1,4,1,2,0,0,0,3,0,6,4,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
9103,1.0,4.0,5.0,1.0,1.0,1.0,2.0,5.0,2.0,5.0,4.0,3.0,5.0,2.0,4.0,4.0,4.0,2.0,5.0,4.0,3,1,1,1,3,3,2,0,5,0,5,2,2,1,6,3,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
9104,1.0,3.0,5.0,1.0,4.0,5.0,1.0,1.0,5.0,4.0,3.0,3.0,5.0,4.0,5.0,1.0,1.0,1.0,1.0,5.0,3,2,2,1,1,6,2,6,0,0,6,2,0,6,6,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
9105,3.0,3.0,5.0,1.0,1.0,5.0,4.0,2.0,3.0,3.0,5.0,5.0,4.0,2.0,3.0,5.0,2.0,3.0,4.0,3.0,2,2,2,1,1,5,2,5,5,0,4,2,0,2,2,3,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


#### 5.데이터 분할

In [11]:
# train에서 label값인 'vote'를 추출하여 feature값인 train_x, label값인 train_y로 분할
train_x = train.drop('voted', axis=1)
train_y = train['voted']

In [12]:
train_x

,QaA,QbA,QcA,QdA,QeA,QfA,QgA,QhA,QiA,QjA,QkA,QlA,QmA,QnA,QoA,QpA,QqA,QrA,QsA,QtA,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,age_group_+70s,age_group_10s,age_group_20s,age_group_30s,age_group_40s,age_group_50s,age_group_60s,gender_Female,gender_Male,race_Arab,race_Asian,race_Black,race_Indigenous Australian,race_Native American,race_Other,race_White,religion_Agnostic,religion_Atheist,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
0,1.0,5.0,5.0,1.0,1.0,1.0,5.0,5.0,2.0,5.0,2.0,5.0,4.0,1.0,5.0,4.0,1.0,1.0,5.0,5.0,3,1,2,1,1,2,2,1,5,2,4,5,4,1,2,2,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1,1.0,5.0,5.0,1.0,1.0,3.0,1.0,4.0,5.0,5.0,2.0,5.0,5.0,1.0,5.0,5.0,2.0,1.0,3.0,5.0,2,2,4,1,1,1,0,2,2,0,6,1,5,5,6,3,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,5.0,1.0,2.0,1.0,3.0,5.0,4.0,3.0,5.0,3.0,5.0,1.0,1.0,5.0,1.0,1.0,2.0,5.0,1.0,4.0,2,1,3,1,1,0,1,0,5,3,6,0,2,1,6,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3,4.0,1.0,1.0,4.0,1.0,3.0,4.0,1.0,5.0,1.0,5.0,1.0,1.0,4.0,4.0,5.0,4.0,4.0,2.0,5.0,3,2,3,1,1,2,2,4,5,0,1,0,2,0,4,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,4.0,5.0,5.0,3.0,1.0,1.0,1.0,2.0,1.0,5.0,3.0,5.0,4.0,2.0,5.0,1.0,4.0,4.0,4.0,5.0,2,1,4,1,1,6,3,6,2,0,6,2,0,6,4,2,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,1.0,5.0,5.0,1.0,1.0,1.0,3.0,3.0,1.0,5.0,3.0,5.0,4.0,3.0,5.0,5.0,1.0,1.0,5.0,5.0,2,1,6,1,1,4,3,3,5,0,1,2,2,0,3,2,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
36421,3.0,4.0,5.0,2.0,1.0,3.0,1.0,3.0,4.0,5.0,4.0,2.0,5.0,2.0,4.0,5.0,3.0,2.0,5.0,5.0,1,2,4,3,1,5,1,1,6,2,5,4,6,3,6,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
36422,2.0,4.0,2.0,1.0,2.0,1.0,4.0,2.0,2.0,4.0,5.0,5.0,4.0,1.0,5.0,2.0,4.0,1.0,5.0,3.0,4,1,6,1,2,1,2,0,5,1,2,1,2,1,5,2,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
36423,5.0,1.0,1.0,3.0,5.0,1.0,4.0,1.0,4.0,2.0,5.0,5.0,4.0,5.0,2.0,1.0,5.0,1.0,2.0,4.0,4,1,4,3,1,4,4,5,4,1,3,2,5,1,5,3,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
